# Background

## Introduction

### Problem

Earth scientists work with huge, 4+ dimensional datasets (3
spatial dimensions, 1 time dimension, and sometimes even more).

-   Example: Air temperature (longitude, latitude, height, time)

-   Example: Surface pressure from "ensemble" of model simulations
    (longitude, latitude, time, ensemble member)

-   Example: Satellite-retrieved radiation (projection x-coordinate,
    projection y-coordinate, wavenumber)
    
### Question

What's the best way to **store** these 4+ dimensional datasets?

-   Spreadsheets? Not enough dimensions.

-   Matrices? No way to annotate "rows", "columns", etc.

### Answer

The [NetCDF](https://en.wikipedia.org/wiki/NetCDF) format
(Network Common Data Form) developed by
[UCAR/Unidata](https://www.unidata.ucar.edu/software/netcdf/) (right
down the road!).

-   Description: Annotated N-dimensional matrices (arrays)

-   File extension: `.nc`

There are other similar data formats
([HDF](https://en.wikipedia.org/wiki/Hierarchical_Data_Format),
[GRIB](https://en.wikipedia.org/wiki/GRIB)), and some software can work
seamlessly with different formats... but **NetCDF is your new best friend**.

## Architecture

NetCDF files have the following features:

-   Global attributes, describing the contents of the NetCDF file. **Attributes are optional.**

-   Named dimensions, describing the dimensions on the arrays contained in the NetCDF file.

-   Named variables, i.e. the data arrays, each with their own dimensions and attributes.

- "Coordinate variables", named variables whose name matches a dimension name. **Coordinate variables are optional.**

Example: A simple file containing [HadCRUT](https://crudata.uea.ac.uk/cru/data/temperature/) land and sea surface temperature data.

In [27]:
%%bash
ncdump -h data/sst.nc

netcdf sst {
dimensions:
	latitude = 36 ;
	longitude = 72 ;
	field_status_string_length = 1 ;
	time = UNLIMITED ; // (2010 currently)
variables:
	float latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:point_spacing = "even" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	float longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "longitude" ;
		longitude:point_spacing = "even" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:units = "days since 1850-1-1 00:00:00" ;
		time:calendar = "gregorian" ;
		time:start_year = 1850s ;
		time:end_year = 2017s ;
		time:start_month = 1s ;
		time:end_month = 6s ;
		time:axis = "T" ;
	float temperature_anomaly(time, latitude, longitude) ;
		temperature_anomaly:long_name = "near_surface_temperature_anomaly" ;
		temperature_anomaly:units = "K" 

NetCDF file formats also have [different "version
numbers"](https://www.unidata.ucar.edu/software/netcdf/docs/faq.html#How-many-netCDF-formats-are-there-and-what-are-the-differences-among-them):


-   NetCDF3 (version 3) [retired in
    2008](https://www.unidata.ucar.edu/software/netcdf/docs/RELEASE_NOTES.html#autotoc_md60).
    Unidata is [currently version
    4](https://www.unidata.ucar.edu/software/netcdf/docs/RELEASE_NOTES.html#autotoc_md0).

-   However version 3 still widespread (scientists are slow to change
    their ways... too many other things to worry about).

-   Some things in NetCDF4 are impossible in NetCDF3 (e.g., multiple unlimited
    dimensions).

-   Weird read/write bugs are sometimes due to version
    incompatibilities.

NetCDF3 is still everywhere, so you may need to use it.

In [25]:
%%bash
ncdump -k data/sst.nc

netCDF-4


In [26]:
%%bash
ncdump -k data/landfracs.nc

classic


# Command-line tools

There's lots of software for working with NetCDF (HDF, GRIB) files.
First, the command-line tools:

- [NCO](http://nco.sourceforge.net/nco.html) (NetCDF Operators).
- [CDO](https://code.mpimet.mpg.de/projects/cdo/wiki/Cdo#Documentation) (Climate Data Operators).

## NCO

- Developed by Unidata, the makers of the NetCDF format.
- Extremely versatile but a bit clunky to learn.
- Tend to be slower.
- Consist of a suite of separate commands:

## CDO


- Developed by the Max Planck Institute for Meteorology.
- Sleeker and more "modern" NetCDf tool.
- Consists of a single command-line command `cdo` but with [hundreds of subcommands](https://code.mpimet.mpg.de/projects/cdo/wiki/Tutorial#Basic-Usage).
- Inclues a game-changer "[operator chaining](https://code.mpimet.mpg.de/projects/cdo/embedded/index.html#x1-120001.2.6)" feature.
- Powered by C++ with built-in parallelization, so this tends to be faster
- The only major caveat: Your data must fit a rigid format, with 2 horizontal dimensions, an optional vertical dimension, and an optional time dimension.
- CDO must be able to infer these dimensions based on the attributes (using the [CF-standards for coordinates](http://cfconventions.org/cf-conventions/cf-conventions.html#coordinate-types)).

The following example contains CDO.

# Python tools

Next, the python tools:

-   [netCDF4](https://unidata.github.io/netcdf4-python/)

-   [xarray](http://xarray.pydata.org/en/stable/)

-   [iris
    "cubes"](https://scitools.org.uk/iris/docs/v1.13.0/userguide/loading_iris_cubes.html) -- this tool is older, less widely used, falling out of favor (xarray was built
    as an [improvement on
    "cubes"](http://xarray.pydata.org/en/stable/getting-started-guide/faq.html#what-other-netcdf-related-python-libraries-should-i-know-about)).
    
-   [cf-python](https://github.com/NCAS-CMS/cf-python) -- this tool is not widely used.

We will focus on the two most widely used tools, netCDF4 and xarray.

## netCDF4

- The more "low-level" tool (requires more lines of code).
- Generally the fastest tool (unless you are using dask on a supercomputer -- see below).
- Used *internally* in the xarray source code.
- **Warning**: Confusingly, "netCDF4" works with both NetCDF versions 3 and 4! 

I recommend using netCDF4 only if you have a very specific reason to use it over xarray (e.g., running calculations on many many small files on a laptop or individual server).

## xarray 

* The more "high-level" tool (requires fewer lines of code).
* The "new kid" on the block but *extremely* powerful.
* Generally slower, unless you are using a supercomputer and combine xarray with [dask-powered parallel computing](http://xarray.pydata.org/en/stable/user-guide/dask.html).

I recommend using xarray most of the time. 

# Other tools

## MATLAB

If possible, I recommend against using [MATLAB](https://www.mathworks.com/products/matlab.html).

MATLAB's syntax is convenient, but python has many useful tools that have no MATLAB equivalent (e.g., dask, xarray, jupyter, metpy). Python is also open source and has a massive community, which helps with debugging code and developing even more useful tools.

If you know MATLAB but don't know python, an REU internship is a great time to learn (that's how I did it!).

If you really need to use MATLAB (for example, you will be working from MATLAB scripts provided by your advisor), more information on MATLAB's NetCDF utilities can be found [here](https://www.mathworks.com/help/matlab/network-common-data-form.html).

## Julia

If possible, I also recommend against the [Julia language](https://julialang.org/) for the time being.

Julia's NetCDF tools are currently not as advanced as python, and the community is relatively small. The Julia language is a really awesome idea, and I'm excited about it! But I don't think Julia has proved itself as being worthy for us to jump ship from python yet.

If you disagree, the two major NetCDF-processing tools in Julia are [NetCDF.jl](https://github.com/JuliaGeo/NetCDF.jl) (for MATLAB-style syntax) and [NCDatasets.jl](https://github.com/Alexander-Barth/NCDatasets.jl) (for python's xarray-style syntax).

## NCL

A final tool I didn't mention is [NCL](https://www.ncl.ucar.edu) (the NCAR Command
Language). This is one of my favorites!

-   MATLAB: Everything is an array.

-   Python: Everything is an object (or dictionary, depending on who you
    ask).

-   NCL: Everything is a NetCDF-formatted dataset. If you're a
    geoscientist, this paradigm is pretty awesome.

Sadly, in 2021 you should **avoid using NCL** for two reasons:

-   NCL [is being
    deprecated](https://www.ncl.ucar.edu/Document/Pivot_to_Python/september_2019_update.shtml)
    (Unidata developers are now focusing on python tools).

-   As "cool" as NCL is, it is very slow... among the slowest tools (see [these
    benchmark results](https://github.com/lukelbd/atmos-benchmarks)).
    
In case you do need to use it, the following example reads XYZT temperature and wind data, then saves YZT "eddy heat
flux".

In [7]:
%%bash
ncdump -h data/fluxes_input.nc

netcdf fluxes_input {
dimensions:
	time = UNLIMITED ; // (200 currently)
	plev = 60 ;
	lat = 18 ;
	lon = 36 ;
variables:
	float time(time) ;
		time:long_name = "time" ;
		time:calendar = "360_day" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:axis = "T" ;
		time:standard_name = "time" ;
	float plev(plev) ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:axis = "Z" ;
		plev:standard_name = "pressure level" ;
	float lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degN" ;
		lat:axis = "Y" ;
		lat:standard_name = "latitude" ;
	float lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degE" ;
		lon:axis = "X" ;
		lon:standard_name = "longitude" ;
	double u(time, plev, lat, lon) ;
		u:long_name = "zonal wind" ;
		u:units = "m/s" ;
	double v(time, plev, lat, lon) ;
		v:long_name = "meridional wind" ;
		v:units = "m/s" ;
	double t(time, plev, lat, lon) ;
		t:long_name = "temperature" ;
		t:units = "K" ;
}


The `fluxes.ncl` script contains the following:

```
; Sample NCL file
; This is a comment
f = addfile("data/fluxes_input.nc", "r")
o = addfile("data/fluxes_output.nc", "c")
t = f->t
v = f->v
tstar = t - conform(t, dim_avg_n(t, 3), (/0, 1, 2/))  ; zonal temperature anomaly
vstar = v - conform(v, dim_avg_n(v, 3), (/0, 1, 2/))  ; zonal meridional-wind anomaly
ehf = dim_avg_n(tstar * vstar, 3)  ; eddy heat flux
copy_VarCoords(t(:, :, :, 0), ehf)
ehf@long_name = "eddy heat flux"
ehf@units = "K*m/s"
o->ehf = ehf
```

Running `ncl fluxes.ncl` will show the following (will not work inside this notebook).
```
Variable: t
Type: double
Total Size: 6912000 bytes
            864000 values
Number of Dimensions: 4
Dimensions and sizes:	[time | 200] x [plev | 60] x [lat | 6] x [lon | 12]
Coordinates: 
            time: [0.25..99.75]
            plev: [8.44375..1004.806]
            lat: [-75..75]
            lon: [-165..165]
Number Of Attributes: 2
  long_name :	temperature
  units :	K

Variable: v
Type: double
Total Size: 6912000 bytes
            864000 values
Number of Dimensions: 4
Dimensions and sizes:	[time | 200] x [plev | 60] x [lat | 6] x [lon | 12]
Coordinates: 
            time: [0.25..99.75]
            plev: [8.44375..1004.806]
            lat: [-75..75]
            lon: [-165..165]
Number Of Attributes: 2
  long_name :	meridional wind
  units :	m/s

Variable: ehf
Type: double
Total Size: 576000 bytes
            72000 values
Number of Dimensions: 3
Dimensions and sizes:	[time | 200] x [plev | 60] x [lat | 6]
Coordinates: 
            time: [0.25..99.75]
            plev: [8.44375..1004.806]
            lat: [-75..75]
Number Of Attributes: 2
  units :	K*m/s
  long_name :	eddy heat flux

```

In [18]:
%%bash
# rm data/fluxes_output.nc 2>/dev/null
# ncl fluxes.ncl
ncdump -h data/fluxes_output.nc

netcdf fluxes_output {
dimensions:
	time = 200 ;
	plev = 60 ;
	lat = 18 ;
variables:
	double ehf(time, plev, lat) ;
		ehf:units = "K*m/s" ;
		ehf:long_name = "eddy heat flux" ;
	float time(time) ;
		time:standard_name = "time" ;
		time:axis = "T" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:calendar = "360_day" ;
		time:long_name = "time" ;
	float plev(plev) ;
		plev:standard_name = "pressure level" ;
		plev:axis = "Z" ;
		plev:units = "hPa" ;
		plev:long_name = "pressure level" ;
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:axis = "Y" ;
		lat:units = "degN" ;
		lat:long_name = "latitude" ;
}
